### Study of the effect of national factors on home prices in the US


**Task:** Using publicly available data on the national factors that impact the supply and demand of homes in the US, build a data science model to study the effect of these variables on home prices.

**source to determine factors:** https://docs.google.com/document/d/185qd57PK-W7LwaaoLT6lD6VIECmU7pKlFfNfr6Q0qDo/edit?usp=sharing

**Approach:** The following variables are chosen for the study:

1. Unemployment Rate
2. Employment Rate
3. Per capita GDP
4. Median Household Income
5. Construction Prices
6. CPI
7. Interest Rates
8. The number of new houses supplied
9. Working Population
10. Urban Population
11. Percentage of population above 65
12. Housing subsidies
13. Number of Households

As a proxy for home prices, the S&P **Case-Shiller Index** is used.

**Note:** Most of the data is downloaded from [https://fred.stlouisfed.org/].

Data for all the variables is downloaded, preprocessed, and combined to create a dataset that can be used for the study. Data for different variables had different frequencies. So, to combine the data, the necessary interpolations are made.


### Importing neccessary libraries

In [198]:
import numpy as np
import pandas as pd

### Preprocessing

In [199]:
# Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("/kaggle/input/usa-home-data/house_price_index_m_c.csv")

# Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])

# # Selecting data till JULY 2023
# mask = df_CS["DATE"] <= "2023-07-01"
# df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print("Shape of the CASE-SHILLER Index:- ", df_CS.shape)
df_CS.tail()


Shape of the CASE-SHILLER Index:-  (289, 4)


,DATE,CSUSHPISA,Year,Month
284,2023-09-01,310.968,2023,9
285,2023-10-01,312.866,2023,10
286,2023-11-01,313.661,2023,11
287,2023-12-01,314.381,2023,12
288,2024-01-01,315.548,2024,1


In [200]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("/kaggle/input/usa-home-data/unemp_rate_m_c.csv")
print("Unemployment Rate Data:- ", df_unemp.shape)
df_unemp.tail()

Unemployment Rate Data:-  (289, 2)


,DATE,UNRATE
284,2023-09-01,3.8
285,2023-10-01,3.8
286,2023-11-01,3.7
287,2023-12-01,3.7
288,2024-01-01,3.7


In [201]:
# Reading Employment Rate Data into a dataframe
df_emp = pd.read_csv("/kaggle/input/usa-home-data/emp_rate_m_c.csv")
df_emp = df_emp.rename(columns={'LREM64TTUSM156S': 'EmpRate'})
print("shape of the Employment Rate Data:- ", df_emp.shape)
df_emp.tail()

shape of the Employment Rate Data:-  (289, 2)


,DATE,EmpRate
284,2023-09-01,72.04477
285,2023-10-01,71.97620
286,2023-11-01,72.09785
287,2023-12-01,71.82301
288,2024-01-01,72.02269


In [202]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("/kaggle/input/usa-home-data/per_cap_gdp_q_c.csv", names = ["DATE", "A939RX0Q048SBEA"], skiprows = 1)
df_pcgdp = df_pcgdp.rename(columns={'A939RX0Q048SBEA': 'Per_Capita_GDP'})
print("Shape of the Per Capita GDP Data:- ", df_pcgdp.shape)
df_pcgdp.tail()

Shape of the Per Capita GDP Data:-  (97, 2)


,DATE,Per_Capita_GDP
92,2023-01-01,66096.0
93,2023-04-01,66357.0
94,2023-07-01,67050.0
95,2023-10-01,67513.0
96,2024-01-01,67646.0


The data is quarterly. need to 


In [203]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("/kaggle/input/usa-home-data/interest_rate_m_c.csv")
print("Shape of the Interest rate data:- ",df_Fed_rate.shape)
df_Fed_rate.tail()

Shape of the Interest rate data:-  (289, 2)


,DATE,FEDFUNDS
284,2023-09-01,5.33
285,2023-10-01,5.33
286,2023-11-01,5.33
287,2023-12-01,5.33
288,2024-01-01,5.33


In [204]:
# Reading Construction Material Data into a dataframe
df_cons_price_index = pd.read_csv("/kaggle/input/usa-home-data/constr_prices_m_c.csv", names = ["DATE", "WPUSI012011"], skiprows = 1)
df_cons_price_index = df_cons_price_index.rename(columns={'WPUSI012011': 'Cons_Material'})
print("Shape of the Construction Material Data:- ", df_cons_price_index.shape)
df_cons_price_index.tail()

Shape of the Construction Material Data:-  (289, 2)


,DATE,Cons_Material
284,2023-09-01,332.098
285,2023-10-01,328.743
286,2023-11-01,326.844
287,2023-12-01,327.644
288,2024-01-01,334.259


In [205]:
# Consumer Price Index
df_CPI = pd.read_csv("/kaggle/input/usa-home-data/cpi_m_c.csv", names = ["DATE", "CPIAUCSL"], skiprows = 1)
df_CPI = df_CPI.rename(columns={'CPIAUCSL': 'CPI'})
print("Shape of the Consumer Price Index:- ", df_CPI.shape)
df_CPI.tail()

Shape of the Consumer Price Index:-  (289, 2)


,DATE,CPI
284,2023-09-01,307.288
285,2023-10-01,307.531
286,2023-11-01,308.024
287,2023-12-01,308.742
288,2024-01-01,309.685


In [206]:
# Monthly new house supply
df_house = pd.read_csv("/kaggle/input/usa-home-data/new_house_supply_m_c.csv", names = ["DATE", "MSACSR"], skiprows = 1)
df_house = df_house.rename(columns={'MSACSR': 'Houses'})
print("Shape of the monthly house supply data:- ", df_house.shape)
df_house.tail()


Shape of the monthly house supply data:-  (289, 2)


,DATE,Houses
284,2023-09-01,7.5
285,2023-10-01,7.9
286,2023-11-01,8.8
287,2023-12-01,8.2
288,2024-01-01,8.3


In [207]:
# Population above 65

df_oldpop = pd.read_csv("/kaggle/input/usa-home-data/pop_abv_65_y_22.csv", names = ["DATE", "old_age_pop"], skiprows = 1)
print("Shape of the population data age above 65:- ", df_oldpop.shape)
df_oldpop.tail()

Shape of the population data age above 65:-  (23, 2)


,DATE,old_age_pop
18,2018-01-01,15.397698
19,2019-01-01,15.791801
20,2020-01-01,16.223400
21,2021-01-01,16.678895
22,2022-01-01,17.128121


In [208]:
# Urban Population Percent

df_urban = pd.read_csv("/kaggle/input/usa-home-data/urban_pop_y.csv")
print("Shape of the urban population percent data:- ", df_urban.shape)
df_urban.tail()

Shape of the urban population percent data:-  (23, 2)


,DATE,Urban Population (% of Total Population)
18,2018-01-01,82.256
19,2019-01-01,82.459
20,2020-01-01,82.664
21,2021-01-01,82.873
22,2022-01-01,83.084


In [209]:
# Housing Subsidies

df_subsidy = pd.read_csv("/kaggle/input/usa-home-data/housing_subsidy_y_22.csv", names = ["DATE", "Subsidy"], skiprows = 1)
print("Shape of the housing subsidies:- ", df_subsidy.shape)
df_subsidy.tail()


Shape of the housing subsidies:-  (23, 2)


,DATE,Subsidy
18,2018-01-01,38.859
19,2019-01-01,40.185
20,2020-01-01,44.147
21,2021-01-01,45.299
22,2022-01-01,48.021


In [210]:
# Working age population

df_working = pd.read_csv("/kaggle/input/usa-home-data/working_pop_m_c.csv", names = ["DATE", "LFWA64TTUSM647S"], skiprows = 1)
df_working = df_working.rename(columns={'LFWA64TTUSM647S': 'working_age_pop'})
print("Shape of the working age population:- ", df_working.shape)
df_working.tail()

Shape of the working age population:-  (289, 2)


,DATE,working_age_pop
284,2023-09-01,209000000.0
285,2023-10-01,208959200.0
286,2023-11-01,209041500.0
287,2023-12-01,209118700.0
288,2024-01-01,208623100.0


In [211]:
# Real Median Household Income

df_income = pd.read_csv("/kaggle/input/usa-home-data/med_houshold_inc_y_22.csv", names = ["DATE", "MEHOINUSA672N"], skiprows = 1)
df_income = df_income.rename(columns={'MEHOINUSA672N': 'median_income'})
print("Shape of the median household income data:- ", df_income.shape)
df_income.tail()


Shape of the median household income data:-  (23, 2)


,DATE,median_income
18,2018-01-01,73030
19,2019-01-01,78250
20,2020-01-01,76660
21,2021-01-01,76330
22,2022-01-01,74580


In [212]:
# Total number of households

df_households = pd.read_csv("/kaggle/input/usa-home-data/total_household_y_23.csv", names = ["DATE", "TTLHH"], skiprows = 1)
df_households = df_households.rename(columns={'TTLHH': 'Num_Households'})
print("Shape of the total households data:- ", df_households.shape)
df_households.tail()


Shape of the total households data:-  (24, 2)


,DATE,Num_Households
19,2019-01-01,128579.0
20,2020-01-01,128451.0
21,2021-01-01,129224.0
22,2022-01-01,131202.0
23,2023-01-01,131434.0


### Concatinating all the dataframes

In [213]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df_CS.head()


,DATE,CSUSHPISA,Year,Month,Per_Capita_GDP
0,2000-01-01,100.552,2000,1,49335.0
1,2000-02-01,101.339,2000,2,NaN
2,2000-03-01,102.126,2000,3,NaN
3,2000-04-01,102.922,2000,4,50109.0
4,2000-05-01,103.678,2000,5,NaN


In [214]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_working, df_house, df_CPI, df_unemp, df_emp, df_cons_price_index, df_Fed_rate]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()


(289, 11)


,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS
DATE,,,,,,,,,,,
2000-01-01,100.552,2000,1,49335.0,178204700.0,4.3,169.3,4.0,74.25278,144.1,5.45
2000-02-01,101.339,2000,2,NaN,178284800.0,4.3,170.0,4.1,74.27563,144.7,5.73
2000-03-01,102.126,2000,3,NaN,178367900.0,4.3,171.0,4.0,74.22861,145.4,5.85
2000-04-01,102.922,2000,4,50109.0,178536800.0,4.4,170.9,3.8,74.51271,145.6,6.02
2000-05-01,103.678,2000,5,NaN,178770300.0,4.4,171.2,4.0,74.08559,144.9,6.27


In [215]:
# Merging yearly dataframes 
yearly = [df_urban, df_households, df_income, df_subsidy, df_oldpop]
for df1 in yearly:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,Urban Population (% of Total Population),Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2000-01-01,100.552,2000,1,49335.0,178204700.0,4.3,169.3,4.0,74.25278,144.1,5.45,79.057,104705.0,67470.0,19.69,12.31763
2000-02-01,101.339,2000,2,NaN,178284800.0,4.3,170.0,4.1,74.27563,144.7,5.73,79.057,104705.0,67470.0,19.69,12.31763
2000-03-01,102.126,2000,3,NaN,178367900.0,4.3,171.0,4.0,74.22861,145.4,5.85,79.057,104705.0,67470.0,19.69,12.31763
2000-04-01,102.922,2000,4,50109.0,178536800.0,4.4,170.9,3.8,74.51271,145.6,6.02,79.057,104705.0,67470.0,19.69,12.31763
2000-05-01,103.678,2000,5,NaN,178770300.0,4.4,171.2,4.0,74.08559,144.9,6.27,79.057,104705.0,67470.0,19.69,12.31763


In [216]:
print(df.shape)

(289, 16)


In [217]:
df.tail(15)

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,Urban Population (% of Total Population),Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2022-11-01,298.402,2022,11,NaN,207585700.0,9.2,298.648,3.6,71.30197,330.369,3.78,83.084,131202.0,74580.0,48.021,17.128121
2022-12-01,297.658,2022,12,NaN,207565400.0,8.6,298.812,3.5,71.63566,326.449,4.10,83.084,131202.0,74580.0,48.021,17.128121
2023-01-01,297.432,2023,1,66096.0,208308100.0,8.3,300.356,3.4,71.69349,327.338,4.33,NaN,131434.0,NaN,NaN,NaN
2023-02-01,297.700,2023,2,NaN,208344100.0,8.4,301.509,3.6,71.78063,330.532,4.57,NaN,131434.0,NaN,NaN,NaN
2023-03-01,298.800,2023,3,NaN,208191500.0,8.1,301.744,3.5,71.87551,331.729,4.65,NaN,131434.0,NaN,NaN,NaN
2023-04-01,300.297,2023,4,66357.0,208299700.0,7.5,303.032,3.4,71.98092,333.366,4.83,NaN,131434.0,NaN,NaN,NaN
2023-05-01,302.476,2023,5,NaN,208534900.0,6.9,303.365,3.7,71.92525,337.473,5.06,NaN,131434.0,NaN,NaN,NaN
2023-06-01,304.323,2023,6,NaN,208635000.0,7.7,304.003,3.6,71.98663,337.336,5.08,NaN,131434.0,NaN,NaN,NaN
2023-07-01,306.440,2023,7,67050.0,208737000.0,7.3,304.628,3.5,71.95162,334.512,5.12,NaN,131434.0,NaN,NaN,NaN


### Check missing values (NAN)

In [218]:
df.isna().sum()

CSUSHPISA                                     0
Year                                          0
Month                                         0
Per_Capita_GDP                              192
working_age_pop                               0
Houses                                        0
CPI                                           0
UNRATE                                        0
EmpRate                                       0
Cons_Material                                 0
FEDFUNDS                                      0
Urban Population (% of Total Population)     13
Num_Households                                1
median_income                                13
Subsidy                                      13
old_age_pop                                  13
dtype: int64

The "Per_Capita_GDP" column has missing values because the data was quarterly. The missing values in the other columns are due to the unavailability of latest data. We will first fill in the missing values in the "Per_Capita_GDP" column using linear interpolation. We will drop the rows with missing values in the other columns. This means that we will use data from 2000 to 2022.


In [219]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [220]:
df.tail(15)

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,Urban Population (% of Total Population),Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2022-11-01,298.402,2022,11,65898.000000,207585700.0,9.2,298.648,3.6,71.30197,330.369,3.78,83.084,131202.0,74580.0,48.021,17.128121
2022-12-01,297.658,2022,12,65997.000000,207565400.0,8.6,298.812,3.5,71.63566,326.449,4.10,83.084,131202.0,74580.0,48.021,17.128121
2023-01-01,297.432,2023,1,66096.000000,208308100.0,8.3,300.356,3.4,71.69349,327.338,4.33,NaN,131434.0,NaN,NaN,NaN
2023-02-01,297.700,2023,2,66183.000000,208344100.0,8.4,301.509,3.6,71.78063,330.532,4.57,NaN,131434.0,NaN,NaN,NaN
2023-03-01,298.800,2023,3,66270.000000,208191500.0,8.1,301.744,3.5,71.87551,331.729,4.65,NaN,131434.0,NaN,NaN,NaN
2023-04-01,300.297,2023,4,66357.000000,208299700.0,7.5,303.032,3.4,71.98092,333.366,4.83,NaN,131434.0,NaN,NaN,NaN
2023-05-01,302.476,2023,5,66588.000000,208534900.0,6.9,303.365,3.7,71.92525,337.473,5.06,NaN,131434.0,NaN,NaN,NaN
2023-06-01,304.323,2023,6,66819.000000,208635000.0,7.7,304.003,3.6,71.98663,337.336,5.08,NaN,131434.0,NaN,NaN,NaN
2023-07-01,306.440,2023,7,67050.000000,208737000.0,7.3,304.628,3.5,71.95162,334.512,5.12,NaN,131434.0,NaN,NaN,NaN


In [221]:
df.isna().sum()

CSUSHPISA                                    0
Year                                         0
Month                                        0
Per_Capita_GDP                               0
working_age_pop                              0
Houses                                       0
CPI                                          0
UNRATE                                       0
EmpRate                                      0
Cons_Material                                0
FEDFUNDS                                     0
Urban Population (% of Total Population)    13
Num_Households                               1
median_income                               13
Subsidy                                     13
old_age_pop                                 13
dtype: int64

In [222]:
df.dropna(inplace = True)


In [223]:
df.isna().sum()

CSUSHPISA                                   0
Year                                        0
Month                                       0
Per_Capita_GDP                              0
working_age_pop                             0
Houses                                      0
CPI                                         0
UNRATE                                      0
EmpRate                                     0
Cons_Material                               0
FEDFUNDS                                    0
Urban Population (% of Total Population)    0
Num_Households                              0
median_income                               0
Subsidy                                     0
old_age_pop                                 0
dtype: int64

In [224]:
df.tail(15)

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,Urban Population (% of Total Population),Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2021-10-01,273.603,2021,10,65648.000000,205337300.0,6.7,276.434,4.5,70.20831,322.120,0.08,82.873,129224.0,76330.0,45.299,16.678895
2021-11-01,277.309,2021,11,65526.666667,205361800.0,5.9,278.799,4.1,70.61938,328.940,0.08,82.873,129224.0,76330.0,45.299,16.678895
2021-12-01,281.506,2021,12,65405.333333,205323500.0,5.6,280.808,3.9,70.83883,335.032,0.08,82.873,129224.0,76330.0,45.299,16.678895
2022-01-01,286.173,2022,1,65284.000000,207063100.0,5.9,282.390,4.0,70.73431,345.742,0.08,83.084,131202.0,74580.0,48.021,17.128121
2022-02-01,291.223,2022,2,65231.666667,206989700.0,6.1,284.535,3.8,70.90482,343.583,0.08,83.084,131202.0,74580.0,48.021,17.128121
2022-03-01,296.523,2022,3,65179.333333,206817900.0,6.9,287.553,3.6,71.23267,345.852,0.20,83.084,131202.0,74580.0,48.021,17.128121
2022-04-01,300.595,2022,4,65127.000000,206929200.0,8.2,288.764,3.7,71.21400,343.786,0.33,83.084,131202.0,74580.0,48.021,17.128121
2022-05-01,303.702,2022,5,65241.000000,207173600.0,8.1,291.359,3.6,71.35808,353.015,0.77,83.084,131202.0,74580.0,48.021,17.128121
2022-06-01,304.456,2022,6,65355.000000,207373200.0,9.9,294.996,3.6,71.22589,349.800,1.21,83.084,131202.0,74580.0,48.021,17.128121


In [225]:
df.isna().sum()

CSUSHPISA                                   0
Year                                        0
Month                                       0
Per_Capita_GDP                              0
working_age_pop                             0
Houses                                      0
CPI                                         0
UNRATE                                      0
EmpRate                                     0
Cons_Material                               0
FEDFUNDS                                    0
Urban Population (% of Total Population)    0
Num_Households                              0
median_income                               0
Subsidy                                     0
old_age_pop                                 0
dtype: int64

In [228]:
print("Shape of the dataframe after preprocessing:- ", df.shape)

Shape of the dataframe after preprocessing:-  (276, 16)


In [229]:
df.to_csv("final_dataset.csv")

## Conclusion

* proressed the dataset to merge different values with different temporal data.
* fixed the NaN values ( missing values )
* used interpolation to convert the gdp data to monthly data.
* deleted the bottom rows since many values in those were missing due to no update on the data bank
* could have techniques liek forward fill or backward fill or advanced time series techniques like ARIMA but it gave less accurate results.